In [3]:
import cobra
from cobra import Model, Reaction, Metabolite
import escher
import escher.urls
import json
import os
from IPython.display import HTML
from cobra.io import load_model
from cobra.flux_analysis.loopless import add_loopless, loopless_solution
from cobra.flux_analysis import pfba

In [1]:
# Download Pseudomonas putida metabolic models
! wget http://bigg.ucsd.edu/static/models/iJN746.xml 
! wget https://www.ebi.ac.uk/biomodels/model/download/MODEL1507180043.2?filename=MODEL1507180043_url.xml

--2022-08-14 11:56:48--  http://bigg.ucsd.edu/static/models/iJN746.xml
Resolving bigg.ucsd.edu (bigg.ucsd.edu)... 169.228.33.117
Connecting to bigg.ucsd.edu (bigg.ucsd.edu)|169.228.33.117|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5116056 (4.9M) [application/xml]
Saving to: 'iJN746.xml'

iJN746.xml          100%[===================>]   4.88M  5.69MB/s    in 0.9s    

2022-08-14 11:56:49 (5.69 MB/s) - 'iJN746.xml' saved [5116056/5116056]

--2022-08-14 11:56:49--  https://www.ebi.ac.uk/biomodels/model/download/MODEL1507180043.2?filename=MODEL1507180043_url.xml
Resolving www.ebi.ac.uk (www.ebi.ac.uk)... 193.62.193.80
Connecting to www.ebi.ac.uk (www.ebi.ac.uk)|193.62.193.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/xml]
Saving to: 'MODEL1507180043.2?filename=MODEL1507180043_url.xml'

MODEL1507180043.2?f     [    <=>             ] 983.84K  1.03MB/s    in 0.9s    

2022-08-14 11:56:51 (1.03 MB/s) - 'MODEL

In [13]:
model = cobra.io.read_sbml_model('Downloads/iJN746.xml')
model2 = cobra.io.read_sbml_model('Downloads/MODEL1507180043_url.xml')
ecoli_model = load_model("iJO1366")

In [17]:
model.reactions.EX_glc__D_e.bounds= (-18,0)
model.reactions.EX_o2_e.bounds=(-20,0)
#model.reactions.EX_tol_e.bounds=(-11.9,0)

In [18]:
nominal = model.optimize()
loopless = loopless_solution(model)

In [19]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
fe2_e,EX_fe2_e,0.0014,0,0.00%
glc__D_e,EX_glc__D_e,8.262,6,87.72%
nh4_e,EX_nh4_e,14.44,0,0.00%
o2_e,EX_o2_e,4.065,0,0.00%
pi_e,EX_pi_e,1.072,0,0.00%
so4_e,EX_so4_e,0.3263,0,0.00%
mhpglu_c,SK_5mthglu_c,0.2744,25,12.14%
dna5mtc_c,SK_dna5mtc_c,0.007,11,0.14%
Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,-2.919,1,28.15%


In [20]:
b = escher.Builder(map_name='iJO1366.Central metabolism',
                   reaction_data=loopless.fluxes,
                   # color and size according to the absolute value
                   reaction_styles=['color', 'size', 'abs', 'text'],
                   # change the default colors
                   reaction_scale=[{'type': 'min', 'color': '#cccccc', 'size': 4},
                                   {'type': 'mean', 'color': '#0000dd', 'size': 20},
                                   {'type': 'max', 'color': '#ff0000', 'size': 40}],
                   # only show the primary metabolites
                   hide_secondary_metabolites=False, 
                   show_gene_reaction_rules=True)
b

Builder(hide_secondary_metabolites=False, reaction_data={'EX_gly_e': 0.0, '3OAR141': 0.008399999999999927, 'EX…

In [21]:
from cobra import Model, Reaction, Metabolite

#https://www.ncbi.nlm.nih.gov/pmc/articles/PMC197239/?page=2

npthl_e = Metabolite(
    'npthl_e',
    formula='C10H8',
    name='Naphthalene',
    compartment='e')

npthl_c = Metabolite(
    'npthl_c',
    formula='C10H8',
    name='Naphthalene',
    compartment='c')

reaction=Reaction('EX_npthl_e')
reaction.name='Naphthalene exchange'
reaction.subsystem='NAH7 pathway'
reaction.lower_bound=-100
reaction.upper_bound=0
reaction.add_metabolites({npthl_e:-1,})
model.add_reaction(reaction)

reaction=Reaction('NPTHLt2')
reaction.name='Naphthalene Transport'
reaction.subsystem='NAH7 pathway'
reaction.lower_bound=-1000
reaction.upper_bound=1000
reaction.add_metabolites({npthl_c:-1,npthl_e:1})
model.add_reaction(reaction)

dhnpthld_c = Metabolite(
    'dhnpthld_c',
    formula='C10H10O2',
    name='1,2-Dihydronaphthalene-1,2-diol',
    compartment='c')



reaction=Reaction('NAPDO')
reaction.name='Naphthalene 1,2-dioxygenase'
reaction.subsystem='NAH7 pathway'
reaction.lower_bound=-1000
reaction.upper_bound=1000
reaction.add_metabolites({model.metabolites.npthl_c:-1,
                          model.metabolites.nadh_c:-1,
                          model.metabolites.h_c:-1,
                          model.metabolites.o2_c:-1,
                          model.metabolites.nad_c:1,
                          dhnpthld_c:1})
model.add_reaction(reaction)


npthld_c = Metabolite(
    'npthld_c',
    formula='C10H8O2',
    name='Naphthalene-1,2-diol',
    compartment='c')

reaction=Reaction('DHDDH')
reaction.name='Dihydrodiol dehydrogenase'
reaction.subsystem='NAH7 pathway'
reaction.lower_bound=-1000
reaction.upper_bound=1000
reaction.add_metabolites({model.metabolites.nad_c:-1,
                          model.metabolites.nadh_c:1,
                          model.metabolites.h_c:1,
                          model.metabolites.dhnpthld_c:-1,
                          npthld_c:1})
model.add_reaction(reaction)

# not in BIGG
hychr2c_c = Metabolite(
    'hychr2c_c',
    formula='C10H7O4',
    name='2-Hydroxychromene-2-carboxylate',
    compartment='c')

reaction=Reaction('DHNDO')
reaction.name='Dihydroxynaphthalene dioxygenase '
reaction.subsystem='NAH7 pathway'
reaction.lower_bound=-1000
reaction.upper_bound=1000
reaction.add_metabolites({model.metabolites.npthld_c:-1,
                          model.metabolites.o2_c:-1,
                          model.metabolites.h_c:1,
                          hychr2c_c:1})
model.add_reaction(reaction)

# not in BIGG
hphobut3en_c= Metabolite(
    'hphobut3en_c',
    formula='C10H7O4',
    name='2-hydroxyphenyl-2-oxobut-3-enoate',
    compartment='c')

reaction=Reaction('HCCA')
reaction.name='2-hydroxychromene-2-carboxylate isomerase'
reaction.subsystem='NAH7 pathway'
reaction.lower_bound=-1000
reaction.upper_bound=1000
reaction.add_metabolites({model.metabolites.hychr2c_c:-1,
                          hphobut3en_c:1})
model.add_reaction(reaction)

# not in BIGG
hbald_c= Metabolite(
    'hbald_c',
    name='2-hydroxybenzaldehyde',
    formula='C7H6O2',
    compartment='c')

reaction=Reaction('tHBPAHA')
reaction.name='Trans-o-hydroxybenzylidenepyruvate hydratase-aldolase'
reaction.subsystem='NAH7 pathway'
reaction.lower_bound=-1000
reaction.upper_bound=1000
reaction.add_metabolites({model.metabolites.hphobut3en_c:-1,
                          model.metabolites.h2o_c:-1,
                          model.metabolites.pyr_c:1,
                          hbald_c:1})
model.add_reaction(reaction)



salc_c = Metabolite(
    'salc_c',
    formula='C7H5O3',
    name='Salicylate',
    compartment='c')

reaction=Reaction('SALDH')
reaction.name='salicylaldehyde dehydrogenase'
reaction.subsystem='NAH7 pathway'
reaction.lower_bound=-1000
reaction.upper_bound=1000
reaction.add_metabolites({model.metabolites.hbald_c:-1,
                          model.metabolites.nad_c:-1,
                          model.metabolites.h2o_c:-1,
                          model.metabolites.nadh_c:1,
                          model.metabolites.h_c:2,
                          salc_c:1})
model.add_reaction(reaction)



reaction=Reaction('SALCOD')
reaction.name= 'Salicylate NADH oxygenoxidoreductase 1-hydroxylating'
reaction.subsystem='NAH7 pathway'
reaction.lower_bound=-1000
reaction.upper_bound=1000


reaction.add_metabolites({
    model.metabolites.h_c:-2,
    model.metabolites.co2_c:1,
    model.metabolites.h2o_c:1, 
    model.metabolites.nad_c:1, 
    model.metabolites.nadh_c:-1,
    model.metabolites.o2_c:-1,
    salc_c:-1,
    model.metabolites.catechol_c:1})
model.add_reaction(reaction)


In [22]:
model.reactions.EX_glc__D_e.bounds= (0,0)
nominal = model.optimize()
loopless = loopless_solution(model)

In [23]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
fe2_e,EX_fe2_e,0.001213,0,0.00%
nh4_e,EX_nh4_e,12.51,0,0.00%
npthl_e,EX_npthl_e,4.944,10,89.16%
o2_e,EX_o2_e,20,0,0.00%
pi_e,EX_pi_e,0.9282,0,0.00%
so4_e,EX_so4_e,0.2826,0,0.00%
mhpglu_c,SK_5mthglu_c,0.2377,25,10.72%
dna5mtc_c,SK_dna5mtc_c,0.006063,11,0.12%
Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,-9.149,1,59.11%


In [24]:
b = escher.Builder(map_name='iJO1366.Central metabolism',
                   reaction_data=loopless.fluxes,
                   # color and size according to the absolute value
                   reaction_styles=['color', 'size', 'abs', 'text'],
                   # change the default colors
                   reaction_scale=[{'type': 'min', 'color': '#cccccc', 'size': 4},
                                   {'type': 'mean', 'color': '#0000dd', 'size': 20},
                                   {'type': 'max', 'color': '#ff0000', 'size': 40}],
                   # only show the primary metabolites
                   hide_secondary_metabolites=False, 
                   show_gene_reaction_rules=True)
b

Builder(hide_secondary_metabolites=False, reaction_data={'EX_gly_e': 0.0, '3OAR141': 0.007276190739519266, 'EX…

In [166]:
model.reactions.NPTHLt2.reaction

'npthl_c <=> npthl_e'

In [159]:
model.reactions.NAPDO.reaction

'h_c + nadh_c + npthl_c + o2_c <=> dhnpthld_c + nad_c'

In [160]:
model.reactions.DHDDH.reaction

'dhnpthld_c + nad_c <=> h_c + nadh_c + npthld_c'

In [161]:
model.reactions.DHNDO.reaction

'npthld_c + o2_c <=> h_c + hychr2c_c'

In [162]:
model.reactions.HCCA.reaction

'hychr2c_c <=> hphobut3en_c'

In [163]:
model.reactions.tHBPAHA.reaction

'h2o_c + hphobut3en_c <=> hbald_c + pyr_c'

In [164]:
model.reactions.SALDH.reaction

'h2o_c + hbald_c + nad_c <=> 2 h_c + nadh_c + salc_c'

In [165]:
model.reactions.SALCOD.reaction

'2 h_c + nadh_c + o2_c + salc_c <=> catechol_c + co2_c + h2o_c + nad_c'

In [127]:
model.reactions.CAT23DOX.reaction

'catechol_c + o2_c --> 2hmcnsad_c'

In [123]:
model.reactions.HMSH.reaction

'2hmcnsad_c + h2o_c --> for_c + 2.0 h_c + op4en_c'

In [185]:
model.reactions.OP4ENH.reaction

'h2o_c + op4en_c --> 4h2opntn_c'

In [186]:
model.reactions.HOPNTAL.reaction

'4h2opntn_c --> acald_c + pyr_c'

In [9]:
model.reactions.ACALD.reaction,loopless.fluxes.ACALD

('acald_c + coa_c + nad_c <=> accoa_c + h_c + nadh_c', 92.13744242259288)

In [10]:
model.reactions.PTAr.reaction,loopless.fluxes.PTAr

('accoa_c + pi_c <=> actp_c + coa_c', 85.18799888140666)

In [11]:
model.reactions.ACKr.reaction,loopless.fluxes.ACKr

('ac_c + atp_c <=> actp_c + adp_c', -85.18799888140666)

In [12]:
model.reactions.ALDD2x_copy2.reaction,loopless.fluxes.ALDD2x_copy2


('acald_c + h2o_c + nad_c <=> ac_c + 2.0 h_c + nadh_c', -88.39294746723655)

In [25]:
model.reactions.ALDD2x_copy2.bounds=(0,0)

In [26]:
nominal = model.optimize()

/home/zeugir/.local/lib/python3.10/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


In [28]:
#DHNDO,HCCA,tHBPAHA,SALDH
napthalene_reactions=['NAPDO','DHDDH','DHNDO','HCCA','tHBPAHA','SALDH',
                      'SALCOD',
               'CAT23DOX','HMSH','OP4ENH','HOPNTAL','ACALD',
               'PTAr','ACKr']

In [31]:
# Calculate Gibbs Free energy of reactions 
from equilibrator_api import ComponentContribution, Q_
from equilibrator_api import Reaction
cc = ComponentContribution()
standard_dg_prime=[]

NAPDO_reaction = Reaction({
    cc.get_compound("kegg:C00829"):-1,
    cc.get_compound("bigg.metabolite:h"):-1,
    cc.get_compound("bigg.metabolite:nadh"):-1,
    cc.get_compound("bigg.metabolite:o2"):-1,
    cc.get_compound("bigg.metabolite:nad"):1,
    cc.get_compound("kegg:C06205"):1})
standard_dg_prime.append(cc.standard_dg_prime(NAPDO_reaction))

for r in napthalene_reactions[1:2]:
    tmp=model.reactions.get_by_id(r).metabolites
    tmp2 = Reaction({cc.get_compound(f"bigg.metabolite:{(str(k).split('_',1)[0])}"):v for k,v in tmp.items()})
    standard_dg_prime.append(cc.standard_dg_prime(tmp2))
    
DHNDO_reaction = Reaction({
    cc.get_compound("bigg.metabolite:npthld"):-1,
    cc.get_compound("bigg.metabolite:o2"):-1,
    cc.get_compound("bigg.metabolite:h"):1,
    cc.get_compound("kegg:C06204"):1})
standard_dg_prime.append(cc.standard_dg_prime(DHNDO_reaction))


HCCA_reaction = Reaction({
    cc.get_compound("kegg:C06204"):-1,
    cc.get_compound("kegg:C06203"):1})
standard_dg_prime.append(cc.standard_dg_prime(HCCA_reaction))

tHBPAHA_reaction = Reaction({
    cc.get_compound("bigg.metabolite:h2o"):-1,
    cc.get_compound("kegg:C06203"):-1,
    cc.get_compound("bigg.metabolite:pyr"):1,
    cc.get_compound("kegg:C06202"):1})
standard_dg_prime.append(cc.standard_dg_prime(tHBPAHA_reaction))

SALDH_reaction = Reaction({
    cc.get_compound("bigg.metabolite:h2o"):-1,
    cc.get_compound("bigg.metabolite:nad"):-1,
    cc.get_compound("kegg:C06202"):-1,
    cc.get_compound("bigg.metabolite:salc"):1,
    cc.get_compound("bigg.metabolite:h"):2,
    cc.get_compound("bigg.metabolite:nadh"):1})
standard_dg_prime.append(cc.standard_dg_prime(SALDH_reaction))

for r in napthalene_reactions[6:]:
    tmp=model.reactions.get_by_id(r).metabolites
    tmp2 = Reaction({cc.get_compound(f"bigg.metabolite:{(str(k).split('_',1)[0])}"):v for k,v in tmp.items()})
    standard_dg_prime.append(cc.standard_dg_prime(tmp2))

In [223]:
{napthalene_reactions[i]: standard_dg_prime[i] for i in range(len(standard_dg_prime))}

{'NAPDO': <Measurement(-340.7110414527393, 8.94220172301959, kilojoule / mole)>,
 'DHDDH': <Measurement(-88.17331630934939, 7.95652863434189, kilojoule / mole)>,
 'DHNDO': <Measurement(-364.12488920485464, 6.800377324646281, kilojoule / mole)>,
 'HCCA': <Measurement(65.30509893913063, 5.544896322929177, kilojoule / mole)>,
 'tHBPAHA': <Measurement(11.807994161697707, 2.059580492552509, kilojoule / mole)>,
 'SALDH': <Measurement(-43.5175704718049, 1.0896565729574628, kilojoule / mole)>,
 'SALCOD': <Measurement(-447.552877793777, 4.936570293334901, kilojoule / mole)>,
 'CAT23DOX': <Measurement(-262.46956871160546, 5.836146439212474, kilojoule / mole)>,
 'HMSH': <Measurement(-44.155352634573546, 1.7599124696207256, kilojoule / mole)>,
 'OP4ENH': <Measurement(0.43160394224855736, 1.4927180898117225, kilojoule / mole)>,
 'HOPNTAL': <Measurement(11.71983755028944, 1.7455924013436208, kilojoule / mole)>,
 'ACALD': <Measurement(-20.67776918543086, 1.2002872522442232, kilojoule / mole)>,
 'PTAr